In [ ]:
!pip install beautifulsoup4

In [480]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [298]:
def get_jobs_links(q):

    #faz a primeira pesquisa para captar o número de páginas resultantes da pesquisa
    response = requests.get(f'https://www.linkedin.com/jobs/search/?currentJobId=3640009990&f_TPR=r2592000&geoId=106057199&keywords={q}&location=Brasil&originalSubdomain=br', verify=False)
    soup = bs(response.text, 'html.parser')

    # recebe o número de resultados da pesquisa e cria uma lista com os parametros necessários para acessar todas as páginas.
    try:
        num_of_results = soup.find(class_ = 'results-context-header__job-count').text
        num_of_pages = int(int(num_of_results)/25)
        range_num_of_pages = list(range(0,num_of_pages))
        start_key_list = [x *25 for x in range_num_of_pages]
        start_key_list
    except ValueError:
        start_key_list = [0,25,50,75,100,125,150,175,200,225]
    
    links = {}
    #acessa cada uma das páginas do resultado da pesquisa
    for key in start_key_list:
        response = requests.get(f'https://www.linkedin.com/jobs/search/?currentJobId=3640009990&f_TPR=r2592000&geoId=106057199&keywords={q}&location=Brasil&originalSubdomain=br&start={key}', verify=False)
        soup = bs(response.text, 'html.parser')
        items = soup.findAll(class_='base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')
        
        
        #verifica se o título do item corresponde a pesquisa
        for item in items:
            job_title = str(item.span.text).strip()
            s = SequenceMatcher(None, q, job_title)
            if s.ratio() > 0.6:
                links[job_title] = item.get('href')
            
    return links 

    


In [299]:
q = 'engenheiro de dados'
list_of_links = get_jobs_links(q)


In [406]:
list_of_links

{'Engenheiro de Dados Júnior': 'https://br.linkedin.com/jobs/view/engenheiro-de-dados-j%C3%BAnior-at-datarisk-3626133619?refId=EE3uDwuQyh03zH8VU3qlsQ%3D%3D&trackingId=W0O0yU4r79ehu7PJR01CUw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'Engenheiro de Dados Jr.': 'https://br.linkedin.com/jobs/view/engenheiro-de-dados-jr-at-h2-club-3619897676?refId=EE3uDwuQyh03zH8VU3qlsQ%3D%3D&trackingId=ICP8XmvK71mHf3veguRxlw%3D%3D&position=5&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'Estágio em Engenharia de Dados': 'https://br.linkedin.com/jobs/view/est%C3%A1gio-em-engenharia-de-dados-at-mirum-agency-3643072174?refId=EE3uDwuQyh03zH8VU3qlsQ%3D%3D&trackingId=NtSzcecPnJn84AuVyJ%2BV6Q%3D%3D&position=10&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'Engenheiro de dados': 'https://br.linkedin.com/jobs/view/engenheiro-de-dados-at-confitec-3635190243?refId=EE3uDwuQyh03zH8VU3qlsQ%3D%3D&trackingId=WosYOTWcS7tZ4mMxLUyFZg%3D%3D&position=7&pageNum=0&trk=public_jobs_jser

In [484]:
def get_dataframe(list_of_links):
    list_of_prequisites = {}
    for link in list(list_of_links.values()):
        response = requests.get(link, verify=False)
        soup = bs(response.text, 'html.parser')
        try:
            list_of_prequisites[link] = soup.find(class_='description__text description__text--rich').find_all('li')
        except(AttributeError):
            pass

    unrepeated_list_of_prequisites = {}
    for key, value in list_of_prequisites.items():
        value = str(value).replace('<li>',' ').replace('</li>',' ')\
        .replace('</li>'," ")\
        .replace('<li>'," ")\
        .replace('['," ")\
        .replace(']'," ")\
        .replace('<br/>'," ")\
        .replace('<br>'," ")\
        .replace('</span>'," ")\
        .replace('<span>'," ")\
        .replace('.'," ")\
        .replace(','," ")\
        .replace(';'," ")\
        .replace(')'," ")\
        .replace('('," ")\
        .replace('/'," ")\
        .lower()\
        .split()
        
        unrepeated_list_of_prequisites[key] = list(set(value))

    stop_words = ['e','a', 'o', 'os', 'do', 'as', 'da', 'em', 'dos', 'se', 'no','com', 'na'\
            ,'para', 'ou', 'e a', 'é', '-', '/', 'de', 'um',' nos', 'que', 'das', 'por'\
            ,'como', '2', 'nos', 'off', 'e em', 'ao', 'com a', 'seu', 'são', 'etc', 'com o'\
            ,'esta', 'área', 'ano', 'dia', 'day','esta', 'nas', 'de', '–', 'à', 'x', 'ser', 'mas', 'sem', 'às', 'uma', ')', 'va', 'in', 'e/ou']


    contagem_de_termos = [[0,0]]
    for value in unrepeated_list_of_prequisites.values():
        for word in value:
            if word not in stop_words:
                if word not in np.array(contagem_de_termos)[:,0:1]:
                    if len(str(word)) > 1:
                        contagem_de_termos.append([word, list(np.concatenate(list(unrepeated_list_of_prequisites.values()))).count(word)])

    df = pd.DataFrame(contagem_de_termos)
    df_result = df.sort_values(by=1, ascending=False).reset_index(drop=True).head(100)

    return df_result

In [ ]:
list_of_prequisites

In [485]:
get_dataframe(get_jobs_links('analista de dados'))

,0,1
0,dados,13
1,bi,10
2,conhecimento,10
3,experiência,9
4,power,9
5,análise,9
6,ferramentas,8
7,problemas,7
8,comunicação,7
9,avançado,6


In [209]:
"""list_of_prequisites = []
for link in list(list_of_links.values()):
    response = requests.get(link, verify=False)
    soup = bs(response.text, 'html.parser')
    try:
        list_of_prequisites.append(soup.find(class_='description__text description__text--rich').find_all('li'))
    except(AttributeError):
        pass
len(list_of_prequisites)"""

"list_of_prequisites = []\nfor link in list(list_of_links.values()):\n    response = requests.get(link, verify=False)\n    soup = bs(response.text, 'html.parser')\n    try:\n        list_of_prequisites.append(soup.find(class_='description__text description__text--rich').find_all('li'))\n    except(AttributeError):\n        pass\nlen(list_of_prequisites)"

In [493]:
list_of_prequisites = {}
for link in list(list_of_links.values()):
    response = requests.get(link, verify=False)
    soup = bs(response.text, 'html.parser')
    try:
        list_of_prequisites[link] = soup.find(class_='description__text description__text--rich').find_all('li')
    except(AttributeError):
        pass
len(list_of_prequisites)

13

In [494]:
list_of_prequisites

{'https://br.linkedin.com/jobs/view/engenheiro-de-dados-j%C3%BAnior-at-datarisk-3626133619?refId=EE3uDwuQyh03zH8VU3qlsQ%3D%3D&trackingId=W0O0yU4r79ehu7PJR01CUw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card': [<li>Atuar como Consultor em projetos de Consultoria, analisando a plataforma de dados, identificando oportunidades e propondo melhorias;</li>,
  <li>Planejar, programar e implantar pipeline de dados no Data Lake (Big Data);</li>,
  <li>Integrar diversos sistemas de APIs de parceiros;</li>,
  <li>Desenvolver, testar e fazer a manutenção de arquiteturas de bancos de dados e sistemas em tempo real ou de forma distribuída;</li>,
  <li>Planejamento e execução em saneamento dos dados.</li>,
  <li>Experiência com ingestão de dados: preparação, tratamento e disponibilização de dados para ambiente analítico;</li>,
  <li>Experiência com linguagem Python e banco de dados SQL;</li>,
  <li>Conhecimento em bibliotecas para análise de dados (Pandas e Spark);</li>,
  <li>Con

In [435]:
unrepeated_list_of_prequisites = {}
for key, value in list_of_prequisites.items():
    value = str(value).replace('<li>',' ').replace('</li>',' ')\
    .replace('</li>'," ")\
    .replace('<li>'," ")\
    .replace('['," ")\
    .replace(']'," ")\
    .replace('<br/>'," ")\
    .replace('<br>'," ")\
    .replace('</span>'," ")\
    .replace('<span>'," ")\
    .replace('.'," ")\
    .replace(','," ")\
    .replace(';'," ")\
    .replace(')'," ")\
    .replace('('," ")\
    .replace('/'," ")\
    .lower()\
    .split()
    
    unrepeated_list_of_prequisites[key] = list(set(value))

In [478]:
stop_words = ['e','a', 'o', 'os', 'do', 'as', 'da', 'em', 'dos', 'se', 'no','com', 'na'\
        ,'para', 'ou', 'e a', 'é', '-', '/', 'de', 'um',' nos', 'que', 'das', 'por'\
        ,'como', '2', 'nos', 'off', 'e em', 'ao', 'com a', 'seu', 'são', 'etc', 'com o'\
        ,'esta', 'área', 'ano', 'dia', 'day','esta', 'nas', 'de', '–', 'à', 'x', 'ser', 'mas', 'sem', 'às', 'uma', ')', 'va', 'in', 'e/ou']


contagem_de_termos = [[0,0]]
for value in unrepeated_list_of_prequisites.values():
    for word in value:
        if word not in stop_words:
            if word not in np.array(contagem_de_termos)[:,0:1]:
                if len(str(word)) > 1:
                    contagem_de_termos.append([word, list(np.concatenate(list(unrepeated_list_of_prequisites.values()))).count(word)])
            
contagem_de_termos

[[0, 0],
 ['saneamento', 1],
 ['atuar', 4],
 ['spark', 5],
 ['options', 1],
 ['parceiros', 2],
 ['integrar', 2],
 ['tratamento', 2],
 ['stock', 1],
 ['gcp', 5],
 ['pipelines', 4],
 ['orquestração', 2],
 ['aws', 7],
 ['planejar', 1],
 ['manutenção', 5],
 ['oracle', 2],
 ['lake', 2],
 ['projetos', 5],
 ['office', 2],
 ['bradesco', 1],
 ['ingestão', 4],
 ['melhorias', 3],
 ['rest', 1],
 ['consultoria', 1],
 ['home', 2],
 ['fazer', 3],
 ['desenvolver', 5],
 ['data', 5],
 ['importante:', 1],
 ['100%', 2],
 ['remoto', 1],
 ['airflow', 7],
 ['processos', 7],
 ['bancos', 4],
 ['aniversário', 2],
 ['odontológico', 3],
 ['real', 1],
 ['preparação', 1],
 ['ambiente', 6],
 ['programar', 1],
 ['pj', 2],
 ['analítico', 1],
 ['dados:', 3],
 ['distribuída', 1],
 ['plataforma', 1],
 ['modelo', 5],
 ['azure', 4],
 ['diversos', 1],
 ['propondo', 2],
 ['execução', 2],
 ['linguagem', 3],
 ['kubernetes', 2],
 ['arquiteturas', 1],
 ['trabalho', 6],
 ['implantar', 1],
 ['testar', 1],
 ['saúde', 4],
 ['banco',

In [482]:
df = pd.DataFrame(contagem_de_termos)
df_result = df.sort_values(by=1, ascending=False).reset_index(drop=True).head(100)
#df[(df[1] >= 3)].sort_values(by=1, ascending=False).reset_index(drop=True).head(50)

,0,1
0,dados,13
1,python,11
2,conhecimento,11
3,sql,11
4,experiência,8
5,airflow,7
6,aws,7
7,processos,7
8,trabalho,6
9,outros,6


In [211]:
#cria uma pasta para a query
#armazenar list_of_prequisites junto com timestamp (nosql)
#armazenar lista de links junto com timestamp (sql)
#recuperar lista de links e tranforma em texto apenas os links que não estiverem na lista
#append lista de links junto com timestamp
#importa todos os textos que estão marcados como a query especificada

In [212]:
"""dict_of_prequisites = {}
for link in list_of_links:
    response = requests.get(link, verify=False)
    soup = bs(response.text, 'html.parser')
    try:
        dict_of_prequisites[link] = (soup.find(class_='description__text description__text--rich').find_all('li'))
    except(AttributeError):
        pass
len(dict_of_prequisites)"""


"dict_of_prequisites = {}\nfor link in list_of_links:\n    response = requests.get(link, verify=False)\n    soup = bs(response.text, 'html.parser')\n    try:\n        dict_of_prequisites[link] = (soup.find(class_='description__text description__text--rich').find_all('li'))\n    except(AttributeError):\n        pass\nlen(dict_of_prequisites)"

In [ ]:
text = str(list_of_prequisites.values()).replace('<li>','').replace('</li>','')\
    .replace('</li>',"")\
    .replace('<li>',"")\
    .replace('[',"")\
    .replace(']',"")\
    .replace('<br/>',"")\
    .replace('<br>',"")\
    .replace('</span>',"")\
    .replace('<span>',"")\
    .replace('.',"")\
    .replace(',',"")\
    .replace(';',"")\
    .lower()
    
words = str(list_of_prequisites.values()).replace('<li>','').replace('</li>','')\
    .replace('</li>',"")\
    .replace('<li>',"")\
    .replace('[',"")\
    .replace(']',"")\
    .replace('<br/>',"")\
    .replace('<br>',"")\
    .replace('</span>',"")\
    .replace('<span>',"")\
    .replace('.',"")\
    .replace(',',"")\
    .replace(';',"")\
    .lower()\
    .split()

stop_words = ['e','a', 'o', 'os', 'do', 'as', 'da', 'em', 'dos', 'se', 'no','com', 'na'\
        ,'para', 'ou', 'e a', 'é', '-', '/', 'de', 'um',' nos', 'que', 'das', 'por'\
        ,'como', '2', 'nos', 'off', 'e em', 'ao', 'com a', 'seu', 'são', 'etc', 'com o'\
        ,'esta', 'área', 'ano', 'dia', 'day','esta', 'nas', 'de', '–', 'à', 'x', 'ser', 'mas', 'sem', 'às', 'uma', ')', 'va', 'in', 'e/ou']
words = set(words)

for sw in stop_words:
    words.discard(sw)
words = list(words)

#a token is a "word" made of two or more words like "data scientist"
tokens = words
for w in range(len(words)):
    if w != len(words)-1:
        w1 = words[w]
        w2 = words[w+1]
        w1_w2 = w1 + ' ' + w2
        tokens.append(w1_w2)
        if w!= len(words)-2:
            w1 = words[w]
            w2 = words[w+1]
            w3 = words[w+2]
            w1_w2_w3 = w1 + ' ' + w2 + ' ' + w3
            tokens.append(w1_w2_w3)
        else:
            pass
    else:
        pass

set(tokens)

In [214]:
contagem_de_termos = []
for token in set(tokens):
    contagem_de_termos.append([token, text.count(token)])

In [296]:
df = pd.DataFrame(contagem_de_termos)
df[(df[1] >= 3)].sort_values(by=1, ascending=False).reset_index(drop=True).head(50)

,0,1
0,r,1340
1,r,1340
2,r,1340
3,r,1340
4,c,786
5,l,668
6,b,210
7,ti,136
8,bi,64
9,bi,64


In [ ]:

#adicionar o título de vaga
#adicionar o termo da pesquisa
#associar o link da publicação com a descrição
#retirar tags
#contar o número de ocorrências de cada palavra
#armazenar o resultado
#rodar o programa duas vezes por dia e adicionar apenas novos registros
#adicionar a data a criação da vaga
#adicionar a data da consulta

In [492]:
def teste(tes):
    a = tes + 'a'
    b = tes + 'b'
    return a, b

testea, testeb = teste('tche')
testeb


'tcheb'

In [496]:

q = 'engenheiro de dados'

link = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?f_TPR=r2592000&geoId=106057199&keywords={q}&location=Brasil&originalSubdomain=br&start=0'
response = requests.get(link, verify=False)
soup = bs(response.text, 'html.parser')
items = soup.findAll(class_='base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]')
for i in items:
    print(i)

<a class="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://br.linkedin.com/jobs/view/t%C3%A9cnico-jr-an%C3%A1lise-de-dados-at-light-3640400990?refId=UoYt22bQ%2FHv4%2FhCPoAoa1Q%3D%3D&amp;trackingId=GRAuJHj9rOUPdpF6iGz6MA%3D%3D&amp;position=1&amp;pageNum=0&amp;trk=public_jobs_jserp-result_search-card">
<span class="sr-only">
              
        
        Técnico Jr (Análise de Dados)
      
      
          </span>
</a>
<a class="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]" data-tracking-client-ingraph="" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-will-navigate="" href="https://br.linkedin.com/jobs/view/engenheiro-de-dados-j%C3%BAnior-at-datarisk-3626133619?refId=UoYt22bQ%2FHv4%2FhCPoAoa1Q%3D%3D&amp;trackingId=9ADEZe92hVjOGP6hHkXOiQ%3D%3D&amp;position=2&amp;page

In [522]:

teste_list = [1,2]
teste_dict = {'title' : 'link', 'title2' : 'link2'}

for key, value in teste_dict.items():
    teste_dict[key] = [value, value + value]
teste_dict.values()

dict_values([['link', 'linklink'], ['link2', 'link2link2']])

In [513]:
for key, list in teste_dict.items():
    print(key)
    for item in list:
        print(item)

title
link
linklink
title2
link2
link2link2


In [528]:
#connect to the database and cehck if query is present to fecth the corresponding data

import psycopg2

# Tem que pesquisar sobre como armazenar esses dados de maneira segura
conn = psycopg2.connect("host=localhost port=5432 dbname=linkedin_scrapping user=adminleo password=fica")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("SELECT * FROM main_table")

# Retrieve query results
records = cur.fetchall()


records

[(None,
  None,
  None,
  'teste',
  datetime.datetime(2023, 6, 30, 17, 11, 41, 173621)),
 (None,
  None,
  None,
  'teste2',
  datetime.datetime(2023, 6, 30, 17, 55, 3, 389836))]

teste
teste2


In [548]:
q = 'teste2'
if q in np.array(records)[:,3:4]:
    print('True')

True


In [8]:
import psycopg2
credentials = "host=localhost port=5432 dbname=linkedin_scrapping user=adminleo password=fica, client_encoding=utf8"
q = 'engenheiro de dados'

def query_descriptions(q):
    conn = psycopg2.connect(credentials)
    cur = conn.cursor()
    cur.execute("SELECT description \
                FROM main_table \
                WHERE q = %s", (q,))
    records = cur.fetchall()
    cur.close()
    conn.close()
    return records


In [11]:
teste = query_descriptions(q)
teste[0]

('[<li>Atuar como Consultor em projetos de Consultoria, analisando a plataforma de dados, identificando oportunidades e propondo melhorias;</li>, <li>Planejar, programar e implantar pipeline de dados no Data Lake (Big Data);</li>, <li>Integrar diversos sistemas de APIs de parceiros;</li>, <li>Desenvolver, testar e fazer a manutenção de arquiteturas de bancos de dados e sistemas em tempo real ou de forma distribuída;</li>, <li>Planejamento e execução em saneamento dos dados.</li>, <li>Experiência com ingestão de dados: preparação, tratamento e disponibilização de dados para ambiente analítico;</li>, <li>Experiência com linguagem Python e banco de dados SQL;</li>, <li>Conhecimento em bibliotecas para análise de dados (Pandas e Spark);</li>, <li>Conhecimento de cloud, construção de pipelines de dados (Oracle, AWS, Azure ou GCP).</li>, <li>Conhecimento em Kubernetes;</li>, <li>Experiência de orquestração de processos com Airflow;</li>, <li>Construção e manutenção de APIs REST.</li>, <li>Tr

In [12]:
def check_if_q_exists(q):
    # Tem que pesquisar sobre como armazenar esses dados de maneira segura
    conn = psycopg2.connect(credentials)
    cur = conn.cursor()
    cur.execute("SELECT q FROM main_table")
    records = cur.fetchall()

    if q in records:
        print('q found in database')
        return True
    else:
        print('q not in database')
        return False

In [14]:
teste = check_if_q_exists(q)

q not in database


In [15]:
teste

False

In [23]:
conn = psycopg2.connect(credentials)
cur = conn.cursor()
cur.execute("SELECT q FROM main_table")
records = [r[0] for r in cur.fetchall()]
records

['teste',
 'teste2',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados',
 'engenheiro de dados']

In [24]:
q in records

True